In [58]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# %pip install import_ipynb
import import_ipynb

In [49]:
numSamples = 204

base_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
dataset = pd.read_csv(base_dir+'\_data\\transformed_dataset.csv')

input = dataset.iloc[:, 4:8]
output = dataset.iloc[:, 8:]
# print(input)
# print(output)
input = input.to_numpy()
output = output.to_numpy()
Xtrain, Xtest, Ytrain, Ytest = train_test_split(input, output, test_size=0.3, train_size=0.7, random_state=0)

### Multi-Linear Regression Implementation with PCA and Stats

In [50]:
from pca import runPCA
pc_x = runPCA(Xtrain, Xtest, 2)
pc_X_train = pc_x[0]
pc_X_test = pc_x[1]
scalingX = pc_x[3]

pc_y = runPCA(Ytrain, Ytest, 2)
pc_Y_train = pc_y[0]
pc_Y_test = pc_y[1]
compY = pc_y[2]
scalingY = pc_y[3]

mlr = LinearRegression()
mlr.fit(pc_X_train, pc_Y_train)

Y_pred_pca = mlr.predict(pc_X_test)

# print("Y_test shape:", Ytest.shape)
# print("Y_pred_pca shape:", Y_pred_pca.shape)

Y_pred_pca_inverse = compY.inverse_transform(Y_pred_pca)
Y_pred = scalingY.inverse_transform(Y_pred_pca_inverse)
print("Test=",Ytest.shape)
print("Pred=",Y_pred.shape)

# print("Y_pred shape after inverse transform:", Y_pred.shape)

Test= (62, 8)
Pred= (62, 8)


### Generate Metrics

In [53]:
mse = mean_squared_error(Ytest, Y_pred)
print("MSE = ", mse)
r2 = r2_score(Ytest, Y_pred)
print("R2 Score = ", r2)

MSE =  0.26263237398322015
R2 Score =  -0.02385333835297762


### Generate Plots

In [61]:
# plt.figure(figsize=(6, 5))
# plt.scatter(Xtest, Ytest, label = 'Test Data')
# plt.scatter(Xtrain, Ytrain, color="green", label = 'Train Data' )
# plt.plot(Xtest, Y_pred, color ="red", label ='Prediction')
# plt.grid(True)
# plt.show()